# My Tools

This Jupyter notebook called "MyTools.ipynb" uses the SageMath 10.0 kernel.

The following code block sets some default options for plotting graphs in SageMath. It sets the default figure size, makes the plots transparent, displays edge labels, uses the spring layout algorithm for graph layouts, and sets the background color for edge labels.


In [1]:
# See https://doc.sagemath.org/html/en/reference/graphs/sage/graphs/graph_plot.html
import sage.graphs.graph_plot

# Set the default figure size for plots
sage.graphs.graph_plot.DEFAULT_PLOT_OPTIONS['figsize'] = [3,3]

# Make the plots transparent
sage.graphs.graph_plot.DEFAULT_PLOT_OPTIONS['transparent'] = True

# Display edge labels in the plots
sage.graphs.graph_plot.DEFAULT_PLOT_OPTIONS['edge_labels'] = False

# Set the background color for edge labels to cyan
sage.graphs.graph_plot.DEFAULT_PLOT_OPTIONS['edge_labels_background'] = 'cyan'

The function "my_graph_show" takes a graph object as input and prints the vertices and edges of the graph, as well as a visualization of the graph. Edge labels are show if "eshow" is set to True, which is the default behavior.

In [2]:
def my_graph_show(G,edge_labels=True):
    print('vertices:',G.vertices(sort=True))
    print('edges:',G.edges(sort=True))
    G.show(edge_labels=edge_labels)

In [3]:
def replace_powers(s,nchar=1):
    result=''
    i=0
    while i<len(s):
        if s[i:i+2]=='^2':
            result+=s[i-nchar:i]
            i+=2
        else:
            result+=s[i]
            i+=1
    return result

In [4]:
def table_multiply(Alpha,Beta):
    AlphaBeta=[]
    for A,B in zip(Alpha,Beta):
        AlphaBeta.append([a*b for a,b in zip(A,B)])
    return AlphaBeta

In [5]:
def repmat(value,n,*args):
    # n rows and m columns
    if args:
        m=args[0]
    else:
        m=n
    return [[value] * m for _ in range(n)]

In [6]:
def zeros(n,*args):
    if args:
        m=args[0]
    else:
        m=n
    return repmat(0,n,m)

In [7]:
def ones(n,*args):
    if args:
        m=args[0]
    else:
        m=n
    return repmat(1,n,m)

The add_vertex_monomials function takes a graph G, as well as optional parameters method and ring. The function creates a new graph H with vertices labeled by monomials. The monomials are chosen based on the number of vertices in G. If the method parameter is set to 'alpha' and the number of vertices in G is less than or equal to 10, the monomials are chosen as alphabetical letters ('a' to 'k'). Otherwise, the monomials are chosen as strings of the form 'a0', 'a1', ..., 'an-1', where n is the number of vertices in G. The function then adds the vertices from G to H using the monomials as labels, and adds the edges from G to H using the monomials as endpoints. If the ring parameter is set to True, the function also creates a polynomial ring V with the chosen monomials and 'invlex' order, and returns both H and V. Otherwise, it returns only H.

In [8]:
def add_vertex_monomials(G,method='integer',ring=False):
    # define a list of possible monomials as strings
    monomials_integer=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
    if method=='alpha' and G.order() <= len(monomials_integer):
        monomials=monomials_integer[:G.order()]  # use only a subset of the monomials
    else: # method='integer' or too many alpha
        monomials=['a%s' %(i) for i in range(G.order())]

    # create a polynomial ring with names as specified monomials
    V=PolynomialRing(ZZ,names=monomials,order='invlex')
    # V = symbols(monomials)

    # create a new graph H of the same type (directed or undirected) as G
    if G.is_directed():
        H = DiGraph()
    else:
        H = Graph()

    # add vertices to H as generators of the polynomial ring V
    if G.get_pos():
        d_pos_G = G.get_pos()
        d_pos_H = dict()
        for v in G.vertices(sort=True):
            H.add_vertex(V.gen(v))
            d_pos_H[V.gen(v)]=d_pos_G[v]
        H.set_pos(d_pos_H)
    else:
        for v in G.vertices(sort=True):
            H.add_vertex(V.gen(v))
            
    # add edges to H while mapping the vertices to their corresponding generators in V
    for e in G.edges(sort=True):
        H.add_edge(V.gen(e[0]),V.gen(e[1]),e[2])

    # optionally inject the variable names as symbols in the current namespace
    if ring:
        # V.inject_variables()
        return (H, V)  # return both the created graph H and the polynomial ring V
        #var_names = [str(var) for var in V]
        #inject_variables(",".join(var_names))
        #return H, var_names  # return both the created graph H and the variable names
    else:
        return H  # only return the created graph H

The function add\_edge\_monomials takes in a graph object G and optional parameters method, edge\_vars, ring, and short\_name.

If method is set to 'integer', the function creates a polynomial ring using the given edge variables and assigns variables to the edges of the graph. The edge variables can be represented either as 'e' followed by the first vertex label or the first and second vertex labels concatenated. If the vertex labels are integers and the short_name parameter is set to True, the edge variables are created using only the first vertex label.

If method is set to 'alpha', the function creates a polynomial ring using the given edge variables and assigns variables to the edges of the graph in reverse order. The number of edge variables used is determined by the size of the graph.

The ring parameter, if set to True, injects the polynomial variables into the global namespace and returns the graph and the polynomial ring. Otherwise, it simply returns the graph.



In [9]:
def add_edge_monomials(G,method='integer',
                       edge_vars=['b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'], # no "a" here!
                       ring=False,short_name=False):

    if method=='integer': # method parameter is set to 'integer'
        if G.vertices(sort=True)[0]==0: # checking if the vertex labels are integers
            if short_name:
                edge_vars = ['e%s' %e[0] for e in G.edges(sort=True)] # create a list of edge variables using the first vertex label
            else:
                edge_vars = ['e%s%s' %(e[0],e[1]) for e in G.edges(sort=True)] # create a list of edge variables using both vertex labels
            E=PolynomialRing(ZZ,names=edge_vars,order='invlex') # create a polynomial ring using the edge variables
        else:
            if short_name:
                edge_vars = ['e%s' %e[0] for e in G.edges(sort=True)] # create a list of edge variables using the first vertex label
                edge_vars = [ ev.replace('a','') for ev in edge_vars ] # remove 'a' from the edge variables
                edge_subscripts = ['%s' %(e[0],e[1]) for e in G.edges(sort=True)] # create a list of edge subscripts using both vertex labels
                edge_subscripts = [ ss.replace('a','') for ss in edge_subscripts ] # remove 'a' from edge subscripts
            else:
                edge_vars = ['e_%s%s' %(e[0],e[1]) for e in G.edges(sort=True)] # create a list of edge variables using both vertex labels
                edge_subscripts = ['%s%s' %(e[0],e[1]) for e in G.edges(sort=True)] # create a list of edge subscripts using both vertex labels
            edge_names = []
            for sub in edge_subscripts:
                edge_names.append('e_{\\mathit{' + sub.replace('e','e_') + '}}') # create a list of latex names for the edge variables
            E=PolynomialRing(ZZ,names=edge_vars,order='invlex') # create a polynomial ring using the edge variables
            E._latex_names = edge_names # assign latex names to the polynomial variables
        gen=0;
        for e in G.edges(sort=True):
            G.add_edge(e[0],e[1],E.gen(gen)) # add an edge with the corresponding variable to the graph
            gen=gen+1
    else: # method parameter is set to 'alpha'
        edge_vars = edge_vars[:G.size()] # get the first G.size() edge variables
        E=PolynomialRing(ZZ,names=edge_vars,order='invlex') # create a polynomial ring using the edge variables
        evars = list(E.gens()[:G.size()]) # create a list of the polynomial variables
        evars.reverse()
        for e in G.edges(sort=True):
            G.add_edge(e[0],e[1],evars.pop()) # add an edge with the corresponding variable to the graph
    if ring:
        # E.inject_variables() # inject the polynomial variables into the global namespace
        return (G, E) # return the graph and the polynomial ring
    else:
        return G # return the graph

In [10]:
def enumerate_allosteric_parameters(G=graphs.HouseGraph(),**kwargs):

    defaultKwargs = { 'method': 'integer', 'verbose': False, 'show': False}
    kwargs = { **defaultKwargs, **kwargs }
    #print(kwargs)

    my_method = kwargs['method']
    if my_method not in ['integer','alpha']:
        raise ValueError("enumerate_allosteric_parameters says: method must be 'integer' or 'alpha'.")

    verbose = kwargs['verbose']
    myshow = kwargs['show']

    if my_method=='alpha':
        (G, V) = add_vertex_monomials(G,method='alpha',ring=True)
        Groot=V.gen(0)
    else:
        Groot=0

    if verbose or myshow:
        G.show(edge_labels=False)

    (BFSVertexList,BFSTree) = G.lex_BFS(tree=True,initial_vertex=Groot)
    (T,E)=add_edge_monomials(BFSTree,method=my_method,ring=True,short_name=True)
    T.set_pos(G.get_pos())

    if verbose or myshow:
        T.show(edge_labels=True)
    if verbose:
        show(E)

    F = [E(0)]
    for v in BFSVertexList[1:]: # every element except the first (the root 0, which has been set to E(0))
        P = T.all_paths(v, BFSVertexList[0], use_multiedges=False, report_edges=True, labels=True)
        P = P[0] # there is only one path, so take the first
        f=0
        for p in P:
            f = f+p[2]
        F.append(f)
    nf = len(F) # this should be equal to G.order()

    if verbose:
        show(F)

    KappaVars = []; KappaNames = []
    for e in E.gens():
        estr = str(e)
        if len(estr)!=1:
            estr=estr.replace('e','')
        KappaVars.append('kappa_'+estr)
        KappaNames.append('\\kappa_{\\mathit{' + estr + '}}')

    if verbose:
        print(KappaVars)
        print(KappaNames)

    EtaProb = zeros(nf); EtaMons = zeros(nf); EtaCoef = zeros(nf)
    for i0 in range(len(F)):
        for i1 in range(len(F)):
            fprod=F[i0]*F[i1]
            EtaProb[i0][i1]=fprod
            EtaMons[i0][i1]=fprod.monomials()
            EtaCoef[i0][i1]=fprod.coefficients()

    if verbose:
        show(table(EtaProb))
        show(table(EtaMons))
        show(table(EtaCoef))

    UniqueMons = []
    for mon in sorted(set(flatten(EtaMons))):
        UniqueMons.append(mon)

    if verbose:
        show(UniqueMons)

    EtaVars = []; EtaNames = []
    for mon in UniqueMons:
        vvar = str(mon)
        vvar = vvar.replace('*', '')

        if my_method=='alpha':
            nchar=1
        else:
            nchar=2
        vvar = replace_powers(vvar,nchar)

        if my_method=='integer':
            vvar = vvar.replace('e','')
        EtaVars.append('eta_'+vvar)
        EtaNames.append('\\eta_{\\mathit{' + vvar + '}}')

    if verbose:
        print(EtaVars)
        print(EtaNames)

    A=PolynomialRing(ZZ,names=KappaVars+EtaVars,order='invlex')
    # A.inject_variables()
    A._latex_names = KappaNames+EtaNames
    # https://ask.sagemath.org/question/8202/how-to-give-latex-names-to-generators-of-polynomial-rings/

    d_vars=dict(zip(list(E.gens())+UniqueMons,list(A.gens())))
    d_vars[E(1)]=1

    if verbose:
        print(d_vars)

    KappaProb = zeros(nf); KappaMons = zeros(nf); KappaCoef = zeros(nf);
    Kappa = repmat(A(1),nf)
    for i0 in range(len(F)):
        for i1 in range(0,i0):
            Kappa[i0][i1]=0
        for i1 in range(i0,len(F)):
            fsum = F[i0]+F[i1]
            KappaProb[i0][i1]=fsum
            KappaMons[i0][i1]=fsum.monomials()
            KappaCoef[i0][i1]=fsum.coefficients()
            for m in range(len(KappaMons[i0][i1])):
                mon = KappaMons[i0][i1][m]
                Kappa[i0][i1]*= d_vars[mon]^KappaCoef[i0][i1][m]
            if i0!=i1:
                Kappa[i0][i1]*=2

    if verbose:
        print(KappaProb)
        print(KappaMons)
        print(KappaCoef)
        print(Kappa)

    Eta = repmat(A(1),nf)
    for i0 in range(len(F)):
        for i1 in range(0, i0):
            Eta[i0][i1] = 0
        for i1 in range(i0, len(F)):
            for m in range(len(EtaMons[i0][i1])):
                mon = EtaMons[i0][i1][m]
                Eta[i0][i1] *= d_vars[mon] ** EtaCoef[i0][i1][m]

    if verbose:
        print(Eta)

    KappaEta = table_multiply(Kappa,Eta)

    if verbose or myshow:
        show(KappaEta)

    return (G, T, KappaEta, A)

In [11]:
def normalize(phi):
    import numpy as np
    phi_sum = np.sum(phi)
    return np.array([np.divide(p,phi_sum) for p in phi])

In [12]:
def random_binary_array(n_states):
    import numpy as np
    import random
    return np.random.randint(2, size=n_states)

In [13]:
def make_random_q(n_states):
    import numpy as np
    import random
    q = np.random.uniform(low=0,high=10,size=n_states)
    q[0]=1
    return q

In [14]:
def pade(phi_list,q_list,x_list):
    import numpy as np

    num = np.zeros(x_list.size)
    den = np.zeros(x_list.size)
    for i, (phi, q) in enumerate(zip(phi_list,q_list)):
        num = np.add(num,[phi*q*x**i for x in x_list])
        den = np.add(den,[q*x**i for x in x_list])
    return np.divide(num,den)

In [15]:
def make_simulated_data(phi,q,xlogmin=-3,xlogmax=3,xnum=20,noise=0):
    import numpy as np
    import random

    if len(q)!=len(phi):
        raise ValueError("make_simulated_data says: phi and q are not the same length.")
    if q[0]!=1:
        print('Warning: Setting q0 to 1.')
        q[0]=1

    x = np.logspace(xlogmin,xlogmax,xnum)
    y = pade(phi,q,x) + np.random.normal(size=x.size, scale=noise)
    return (x,y)

In [16]:
def dimerize(phi):
    n = len(phi)
    Phi = zeros(n)
    for i0 in range(n):
        for i1 in range(i0,n):
            Phi[i0][i1] = phi[i0]*phi[i1]
            if i0==i1:
                Phi[i0][i1]*=2
    return Phi

In [17]:
def make_symbolic_dimer_binding_curve(Phi,KappaEta):
    n=len(Phi)
    num=0; den = 0;
    for i0 in range(n):
        for i1 in range(i0,n):
            num += Phi[i0][i1]*KappaEta[i0][i1]
            den += KappaEta[i0][i1]
    return num/den